In [182]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.datasets import mnist
import matplotlib.pyplot as plt
from network import preprocess_data
import numpy as np
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
def preprocess_data(x: np.ndarray, y: np.ndarray, limit: int): 
    # Shuffle the data
    x_train, _, y_train, _ = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    
    x_train = tf.image.convert_image_dtype(tf.expand_dims(x_train[:limit], axis=-1), dtype='float32') 
    y_train = y_train[:limit]
    
    
    y_train = tf.one_hot(y_train, depth=len(np.unique(y_train)), on_value=1, off_value=0)
    print(x_train.shape)
    print(y_train.shape)
    
    return x_train, y_train

In [184]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

np.random.seed(1)
# Limit the data to 2000 and one-hot encoded y
x_train, y_train = preprocess_data(x_train, y_train, 8000)
x_test, y_test = preprocess_data(x_test, y_test, 5000)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

(8000, 28, 28, 1)
(8000, 10)
(5000, 28, 28, 1)
(5000, 10)


In [185]:
def cnn_net(input_shape=(28, 28, 1), n_class=10): 
    input_img = Input(shape=input_shape)
    # CNN's block 1
    Z1 = Conv2D(filters=10, kernel_size=3, strides=1, kernel_initializer='he_normal')(input_img)
    P1 = MaxPooling2D(pool_size=2, strides=2)(Z1)
    A1 = Activation(activation='relu')(P1)
    # CNN's block 2
    Z2 = Conv2D(filters=5, kernel_size=2, strides=1, kernel_initializer='he_normal')(A1)
    P2 = MaxPooling2D(pool_size=2, strides=2)(Z2)
    A2 = Activation(activation='relu')(P2)
    # Dense layer
    F = Flatten()(A2)
    A3 = Dense(units=n_class, kernel_initializer='he_normal', activation='softmax')(F)
    
    model = Model(inputs=input_img, outputs=A3)
    return model

In [186]:
net = cnn_net()
net.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_36 (Conv2D)          (None, 26, 26, 10)        100       
                                                                 
 max_pooling2d_34 (MaxPooli  (None, 13, 13, 10)        0         
 ng2D)                                                           
                                                                 
 activation_34 (Activation)  (None, 13, 13, 10)        0         
                                                                 
 conv2d_37 (Conv2D)          (None, 12, 12, 5)         205       
                                                                 
 max_pooling2d_35 (MaxPooli  (None, 6, 6, 5)           0         
 ng2D)                                                    

In [187]:
learning_rate = 0.015  # You can adjust these values
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

net.compile(optimizer='adam', loss=loss_fn,
            metrics=['accuracy'])

In [188]:
history = net.fit(train_dataset, epochs=50)

Epoch 1/50
125/125 [==============================] - 2s 12ms/step - loss: 0.3690 - accuracy: 0.3070
Epoch 2/50
125/125 [==============================] - 1s 9ms/step - loss: 0.1793 - accuracy: 0.7215
Epoch 3/50
125/125 [==============================] - 1s 9ms/step - loss: 0.1052 - accuracy: 0.8480
Epoch 4/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0821 - accuracy: 0.8851
Epoch 5/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0700 - accuracy: 0.9046
Epoch 6/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0621 - accuracy: 0.9160
Epoch 7/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0565 - accuracy: 0.9251
Epoch 8/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0521 - accuracy: 0.9321
Epoch 9/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0485 - accuracy: 0.9370
Epoch 10/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0456 - accuracy: 0.941

In [189]:
loss, accuracy = net.evaluate(test_dataset)
print('Loss', loss)
print('Accuracy', accuracy*100, '%')

79/79 [==============================] - 1s 6ms/step - loss: 0.0299 - accuracy: 0.9672
Loss 0.029924429953098297
Accuracy 96.7199981212616 %
